In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-10-23 20:21:05.041145: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 20:21:05.041244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 20:21:05.171110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
pd.set_option('display.max_rows', None)  # To display all rows 
pd.set_option('display.max_columns', None)  # To display all columns

In [3]:
# Paths to datasets
train_dir = '/kaggle/input/retinal-disease-classification/Training_Set/Training_Set/Training'
val_dir = '/kaggle/input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation'
test_dir = '/kaggle/input/retinal-disease-classification/Test_Set/Test_Set/Test'

train_labels_path = '/kaggle/input/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv'
val_labels_path = '/kaggle/input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv'
test_labels_path = '/kaggle/input/retinal-disease-classification/Test_Set/Test_Set/RFMiD_Testing_Labels.csv'

In [4]:
# Load labels
train_labels = pd.read_csv(train_labels_path)
val_labels = pd.read_csv(val_labels_path)
test_labels = pd.read_csv(test_labels_path)

In [5]:
# Print dataset shapes
print("Train labels shape:", train_labels.shape)
print("Validation labels shape:", val_labels.shape)
print("Test labels shape:", test_labels.shape)

Train labels shape: (1920, 47)
Validation labels shape: (640, 47)
Test labels shape: (640, 47)


In [8]:
# Data Preprocessing
def load_images_and_labels(image_dir, labels_df):
    images = []
    labels = []
    for index, row in labels_df.iterrows():
        img_path = os.path.join(image_dir, f"{row['ID']}.png")
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224)) # Resize images to 224x224
            images.append(img)
            labels.append(row['Disease_Risk'])
    return np.array(images), np.array(labels)

# Load images and labels
X_train, y_train = load_images_and_labels(train_dir, train_labels)
X_val, y_val = load_images_and_labels(val_dir, val_labels)
X_test, y_test = load_images_and_labels(test_dir, test_labels)

In [9]:
# Normalize the images
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [10]:
# Image Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [11]:
# Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [12]:
# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=20
)

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.

2024-10-23 20:27:26.730528: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49284: 7.84353, expected 6.92193
2024-10-23 20:27:26.730584: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49286: 7.76792, expected 6.84632
2024-10-23 20:27:26.730597: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49287: 6.15631, expected 5.23472
2024-10-23 20:27:26.730607: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 49288: 6.71859, expected 5.797
2024-10-23 20:27:26.730625: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4

 1/60 ━━━━━━━━━━━━━━━━━━━━ 12:42 13s/step - accuracy: 0.3750 - loss: 0.6970

I0000 00:00:1729715253.221938     101 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


60/60 ━━━━━━━━━━━━━━━━━━━━ 31s 302ms/step - accuracy: 0.6934 - loss: 0.9695 - val_accuracy: 0.7906 - val_loss: 0.5773
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 276ms/step - accuracy: 0.7856 - loss: 0.5096 - val_accuracy: 0.7906 - val_loss: 0.4932
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 275ms/step - accuracy: 0.7993 - loss: 0.4738 - val_accuracy: 0.7906 - val_loss: 0.4596
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 280ms/step - accuracy: 0.8068 - loss: 0.4487 - val_accuracy: 0.7906 - val_loss: 0.4559
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 276ms/step - accuracy: 0.7812 - loss: 0.4836 - val_accuracy: 0.7906 - val_loss: 0.4473
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 280ms/step - accuracy: 0.7852 - loss: 0.4570 - val_accuracy: 0.7906 - val_loss: 0.4463
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 275ms/step - accuracy: 0.7937 - loss: 0.4499 - val_accuracy: 0.7906 - val_loss: 0.4734
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 278ms/step - accuracy: 0.7963 - loss: 0.4603 - val_accuracy: 0.790

In [17]:
# Save the model
model.save('/kaggle/working/retinal_disease_classification_model.h5')